In [19]:
import os
import math
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output


def numpy_to_latex_bmatrix(array):
    if len(array.shape) != 2:
        raise ValueError("Input must be a 2D numpy array")
    
    latex_str = "\\begin{bmatrix}"
    for row in array:
        latex_str += " & ".join(map(str, row)) + r" \\"
    latex_str += r"\end{bmatrix}"
    
    return latex_str


def vec2quat(vector):
    """
    returns quaternion which will rotate vector along the most
    efficient path to align with the 3rd-axis [0, 0, 1]
    """
    vector = vector / np.linalg.norm(vector)
    z_axis = np.array([0, 0, 1])
    axis = np.cross(z_axis, vector)
    angle = np.arccos(np.clip(np.dot(z_axis, vector), -1.0, 1.0))  # Clip for stability
    quaternion = R.from_rotvec(axis * angle).as_quat()
    return quaternion, axis, angle


def vec2quat_batch(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    unit_vectors = vectors / norms
    z_axis = np.array([0, 0, -1])
    axes = np.cross(z_axis, unit_vectors)
    angles = np.arccos(np.clip(np.einsum('ij,j->i', unit_vectors, z_axis), -1.0, 1.0))
    sin_half_angles = np.sin(angles / 2)[:, np.newaxis]
    cos_half_angles = np.cos(angles / 2)
    quaternions = np.hstack((cos_half_angles[:, np.newaxis], sin_half_angles * axes))
    return quaternions


def plot_path(df):
    """
    Plots the 3D path of a point over time using Plotly.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing
    x, y, and z positions in the first three columns.
    """
    fig = go.Figure()
    fig.data = []

    # Add trace for the path
    fig.add_trace(go.Scatter3d(
        x=df.iloc[:, 0],  # x positions
        y=df.iloc[:, 1],  # y positions
        z=df.iloc[:, 2],  # z positions
        mode='lines',
        name='Path'
    ))

    # Add markers for start and end points
    fig.add_trace(go.Scatter3d(
        x=[df.iloc[0, 0]],  # start x position
        y=[df.iloc[0, 1]],  # start y position
        z=[df.iloc[0, 2]],  # start z position
        mode='markers',
        marker=dict(size=10, color='green'),
        name='Start'
    ))

    fig.add_trace(go.Scatter3d(
        x=[df.iloc[-1, 0]],  # end x position
        y=[df.iloc[-1, 1]],  # end y position
        z=[df.iloc[-1, 2]],  # end z position
        mode='markers',
        marker=dict(size=10, color='red'),
        name='End'
    ))

    # Update layout
    fig.update_layout(
        title='3D Path Over Time',
        scene=dict(
            xaxis_title='X Position',
            yaxis_title='Y Position',
            zaxis_title='Z Position'
        ),
        showlegend=True
    )

    fig.show()


def plot_trc(fp):
    testtrc = pd.read_csv(fp, sep='\t', skiprows = 3)
    newtrccols = [f"{i.lstrip('Mobility_markerset:')}_{axis}" for i in testtrc.columns[2::3] for axis in ['x', 'y', 'z']]
    testtrc.columns = list(testtrc.columns[:2])+list(newtrccols)
    testtrc.iloc[1:, 2:] = testtrc.iloc[1:, 2:].astype(float) / 10 # Convert mm to cm
    t = testtrc['Time'][1:].to_numpy()

    # Create the figure
    fig = go.Figure()

    # Initialize frames list
    frames = []

    # Loop to add multiple traces
    for i in range(36):
        x = testtrc.iloc[1:, 2 + 3 * i].to_numpy(dtype=float)
        z = testtrc.iloc[1:, 3 + 3 * i].to_numpy(dtype=float)
        y = testtrc.iloc[1:, 4 + 3 * i].to_numpy(dtype=float)
        
        # Add the trace for the path of each set of data
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=1)))

    # Create frames for all traces
    frames = [
        go.Frame(
            data=[
                go.Scatter3d(x=[testtrc.iloc[k, 2 + 3 * i]], y=[testtrc.iloc[k, 4 + 3 * i]], z=[testtrc.iloc[k, 3 + 3 * i]], mode='markers', marker=dict(size=2))
                for i in range(36)
            ],
            name=str(k)
        )
        for k in range(1, len(t), 10)
    ]

    # Determine the axis ranges (adjust as needed)
    x_min, x_max = -500, 200
    y_min, y_max = -50, 200
    z_min, z_max = 0, 200

    # Calculate aspect ratios
    x_range = x_max - x_min
    y_range = y_max - y_min
    z_range = z_max - z_min
    # Calculate the aspect ratio
    max_range = max(x_range, y_range, z_range)
    aspect_ratio = dict(x=x_range / max_range, y=y_range / max_range, z=z_range / max_range)

    camera = dict(
        eye=dict(x=.1, y=0.5, z=.1),  # Adjust these values to set the initial zoom level
        up=dict(x=0, y=0, z=1),  # Adjust these values to set the up direction
        center=dict(x=0, y=0, z=0)  # Adjust these values to set the center of the view
    )

    # Update layout with animation settings and fixed axis ranges
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X Axis', range=[x_min, x_max], autorange=False),
            yaxis=dict(title='Y Axis', range=[y_min, y_max], autorange=False),
            zaxis=dict(title='Z Axis', range=[z_min, z_max], autorange=False),
            aspectratio=aspect_ratio,
            camera=camera
        ),
        updatemenus=[dict(
            type='buttons',
            showactive=False,
            buttons=[
                dict(label='Play', method='animate', args=[None, dict(frame=dict(duration=80, redraw=True), fromcurrent=True)]),
                dict(label='Pause', method='animate', args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])
            ]
        )],
        sliders=[{
            'steps': [{'args': [[f.name], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                    'label': str(i),
                    'method': 'animate'} for i, f in enumerate(frames)],
            'transition': {'duration': 0},
            'x': 0.1,
            'len': 0.9
        }],
        title=f'3D Animation of {fp[13:-4]} markers'
    )

    # Add frames to the figure
    fig.frames = frames

    # Show the plot
    fig.show()


class imuData:
    def __init__(self, name, df, sensor_num):
        self.name = name
        self.indices = [row for row in df.index]

        self.start_row_index = self.indices.index(
            'DelsysTrignoBase 1: Sensor '+str(sensor_num)+'IM ACC Pitch')

        self.all_data = df.iloc[
            self.start_row_index : self.start_row_index+6]

        acc_data = self.all_data.iloc[0:3]
        self.a_local = acc_data.transpose()
        self.a_local.columns.values[0:3] = ['a_pitch', 'a_roll', 'a_yaw']

        sqrt_acc = np.square(self.a_local)
        net_acc_sq = sqrt_acc.apply(np.sum, axis=1, raw=True)
        self.net_acc = np.sqrt(net_acc_sq)

        gyr_data = self.all_data.iloc[3:7]
        self.omega_local = gyr_data.transpose()
        self.omega_local.columns.values[0:3] = ['omega_pitch', 'omega_roll',
                                                'omega_yaw']

        self.measurements = self.a_local.join(self.omega_local)

        self.frames = len(acc_data.columns)

    def __str__(self):
        return f"ImuData object.\nname:  '{self.name}'\nframes: {self.frames}"


class InitialOrientationEstimator:
    def __init__(self, gravity=9.8, noise_acc=0.2, noise_mag=0.1):
        self.gravity = gravity
        self.noise_acc = noise_acc
        self.noise_mag = noise_mag
        self.orientation_prior = None

    def update_prior(self, accelerometer, magnetometer):
        """
        Bayesian update of the orientation prior using a single measurement.
        """
        g = np.array([0, 0, self.gravity])  # Gravity in NED frame

        # Normalize accelerometer and magnetometer vectors
        acc_norm = accelerometer / np.linalg.norm(accelerometer)
        mag_norm = magnetometer / np.linalg.norm(magnetometer)

        # Build orientation candidates from acc and mag
        r_acc = R.align_vectors([acc_norm], [g])[0]  # Align acc to gravity
        north_vector = np.cross(g, acc_norm)
        if np.linalg.norm(north_vector) < 1e-6:
            # Handle edge case: acc aligned with gravity
            north_vector = np.array([1, 0, 0])  # Default north direction
        else:
            north_vector /= np.linalg.norm(north_vector)

        # Align magnetometer with the horizontal plane
        r_mag = R.align_vectors([mag_norm], [north_vector])[0]

        # Combine the rotations (acc->gravity, then align magnetic field)
        candidate_orientation = r_mag * r_acc

        # Update the prior
        if self.orientation_prior is None:
            self.orientation_prior = {
                "mean": candidate_orientation.as_quat(),
                "covariance": np.eye(4) * 1e6  # High uncertainty for uniform prior
            }
        else:
            # Perform the Bayesian update
            mean_prior = R.from_quat(self.orientation_prior["mean"])
            cov_prior = self.orientation_prior["covariance"]
            cov_meas = np.eye(4) * self.noise_acc

            # Kalman gain
            K = cov_prior @ np.linalg.inv(cov_prior + cov_meas)

            # Updated mean and covariance
            mean_updated = mean_prior.as_quat() + K @ (
                candidate_orientation.as_quat() - mean_prior.as_quat()
            )
            cov_updated = (np.eye(4) - K) @ cov_prior

            # Normalize quaternion
            mean_updated /= np.linalg.norm(mean_updated)

            # Store updated values
            self.orientation_prior = {"mean": mean_updated, "covariance": cov_updated}

    def get_estimated_orientation(self):
        if self.orientation_prior is None:
            raise ValueError("Orientation estimation not initialized.")
        return self.orientation_prior["mean"]

# Example Usage
gravity = 9.8
estimator = InitialOrientationEstimator(gravity=gravity)

# Simulated accelerometer and magnetometer measurements
simulated_data = [
    {"acc": np.array([0, 0, 9.8]), "mag": np.array([1, 0, 0])},
    {"acc": np.array([0.1, -0.1, 9.7]), "mag": np.array([1, 0.2, -0.1])},
    # Add more simulated measurements
]

for data in simulated_data:
    estimator.update_prior(data["acc"], data["mag"])

initial_orientation = estimator.get_estimated_orientation()
print("Estimated Initial Orientation (quaternion):", initial_orientation)

    

class BayesianCalibration:
    def __init__(self, gravity=9.8, std_noise=0.1):
        self.gravity = gravity
        self.std_noise = std_noise
        self.weights = None  # To store weights based on net acceleration
        self.quaternions = []  # To store candidate quaternions

    def quaternion_to_gravity(self, q):
        """
        Convert a quaternion to a gravity vector.
        """
        q0, q1, q2, q3 = q
        C = np.array([
            [1 - 2 * (q2**2 + q3**2), 2 * (q1 * q2 - q0 * q3), 2 * (q1 * q3 + q0 * q2)],
            [2 * (q1 * q2 + q0 * q3), 1 - 2 * (q1**2 + q3**2), 2 * (q2 * q3 - q0 * q1)],
            [2 * (q1 * q3 - q0 * q2), 2 * (q2 * q3 + q0 * q1), 1 - 2 * (q1**2 + q2**2)]
        ])
        return C @ np.array([0, 0, -self.gravity])

    def weight_measurement(self, accel):
        """
        Compute a weight for the measurement based on proximity to 9.8 m/s^2.
        """
        net_accel = np.linalg.norm(accel)
        error = abs(net_accel - self.gravity)
        return np.exp(-error**2 / (2 * self.std_noise**2))

    def update(self, accel_data):
        """
        Perform Bayesian updates to estimate the initial orientation.
        """
        # Initialize weights and candidate quaternions
        self.weights = np.zeros(len(accel_data))
        self.quaternions = []  # Placeholder for quaternion candidates
        
        # Generate candidate quaternions (could use grid sampling or optimization)
        for q in self.generate_candidate_quaternions():
            gravity_estimate = self.quaternion_to_gravity(q)
            for i, accel in enumerate(accel_data):
                error = np.linalg.norm(accel - gravity_estimate)
                self.weights[i] += self.weight_measurement(accel)
            self.quaternions.append(q)

        # Normalize weights to form a probability distribution
        self.weights /= np.sum(self.weights)

        # Compute the weighted average quaternion
        weighted_quat = sum(w * np.array(q) for w, q in zip(self.weights, self.quaternions))
        return self.normalize_quaternion(weighted_quat)

    def generate_candidate_quaternions(self, resolution=10):
        """
        Generate a set of candidate quaternions for calibration.
        """
        # Uniform sampling on a quaternion hypersphere
        candidates = []
        for _ in range(resolution):
            q = np.random.randn(4)  # Sample random numbers
            candidates.append(self.normalize_quaternion(q))
        return candidates

    @staticmethod
    def normalize_quaternion(q):
        return q / np.linalg.norm(q)


class KF6:
    def __init__(self, dt, Q, R, x_0, P_0):
        
        self.dt = float(dt)
        
        Q = np.array(Q)
        if Q.shape != (16, 16):
            raise ValueError(
                '"Q" shape error: expected (16, 16), '
                f'got {Q.shape} instead'
            )
        self.Q = Q
        
        R = np.array(R)
        if R.shape != (6, 6):
            raise ValueError(
                '"R" shape error: expected (6, 6), '
                f'got {R.shape} instead'
            )
        self.R = R
        
        x_0 = np.array(x_0)
        if x_0.ndim != 2 or x_0.shape != (16, 1):
            raise ValueError(
                '"x_0" shape error: '
                'expected 2D column vector (16, 1), '
                f'got {x_0.shape} instead'
            )
        self.x = x_0
        
        P_0 = np.array(P_0)
        if P_0.shape != (16, 16):
            raise ValueError(
                '"P_0" shape error: expected (16, 16), '
                f'got {P_0.shape} instead'
            )
        self.P = P_0

    def get_A(self, x, dt):
        wN, wE, wD = [float(x[i, 0]) for i in range(13, 16)]
        A = np.diag(16*[1])
        for i in range(6):
            A[i,i+3] = dt
        bottom = np.array(
            [[1, -dt*wN/2, -dt*wE/2, -dt*wD/2],
            [dt*wN/2, 1, dt*wD/2, -dt*wE/2],
            [dt*wE/2, -dt*wD/2, 1, dt*wN/2],
            [dt*wD/2, dt*wE/2, -dt*wN/2, 1]],
            dtype=float
            )
        A[9:13,9:13] = bottom
        return A

    def quat2matrix(self, q):
        q0, q1, q2, q3 = [float(q[i,0]) for i in range(4)]
        C = np.array(
            [[1 - 2*(q2**2 + q3**2), 2*(q1*q2 - q0*q3),
              2*(q1*q3 + q0*q2)]
            ,[2*(q1*q2 + q0*q3), 1 - 2*(q1**2 + q3**2),
              2*(q2*q3 - q0*q1)]
            ,[2*(q1*q3 - q0*q2), 2*(q2*q3 + q0*q1),
              1 - 2*(q1**2 + q2**2)]]
            )
        return C.reshape(3,3)
    
    def get_H(self, x):
        C = self.quat2matrix(x[9:13])
        H = np.zeros((6, 16))
        H[0:3, 6:9] = C.T
        H[3:6, 13:16] = C.T
        return H
    
    def quat_norm(self, q):
        norm = np.linalg.norm(q)
        if norm == 0:
            raise ValueError('Cannot normalize a zero vector')
        return q/norm

    def predict(self):
        self.A = self.get_A(self.x, self.dt)
        self.H = self.get_H(self.x)
        self.xp = self.A @ self.x
        self.xp[5] -= 9.8*self.dt
        self.xp[9:13] = self.quat_norm(self.xp[9:13])
        self.Pp = self.A @ self.P @ self.A.T + self.Q

    def update(self, z):

        z = np.array(z).reshape(-1, 1)

        self.y = z - self.H @ self.xp
        self.K = self.Pp @ self.H.T @ np.linalg.inv(
            self.H @ self.Pp @ self.H.T + self.R)
        self.x = self.xp + self.K @ (self.y)
        self.P = (np.eye(16) - self.K @ self.H) @ self.Pp
        return self.x


# Get the list of participants (subdirectories)
participants = [d for d in os.listdir('trialData')]
participants.sort()

# Create a dropdown widget for participants
participant_dropdown = widgets.Dropdown(
    options=participants,
    description='Participant:',
    disabled=False,
)

# Create a dropdown widget for files
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    disabled=False,
)

# Update the file dropdown based on the selected participant
def update_files(*args):
    files = [f.rstrip('.csv') for f in os.listdir(f'trialData/{participant_dropdown.value}')]
    files.sort()
    file_dropdown.options = files
    file_dropdown.value = files[0]

participant_dropdown.observe(update_files, 'value')

# Display the widgets
display(participant_dropdown)

# Initialize the file dropdown
update_files()

# Display the file dropdown after it has been populated
display(file_dropdown)

output = widgets.Output()
display(output)

global testdata
def update_output(*args):
    csv_path = f'trialData/{participant_dropdown.value}/{file_dropdown.value}.csv'
    testdf = pd.read_csv(csv_path, index_col=0)
    testimu = imuData('test', testdf, 4)
    testdata = testimu.measurements
    testdata.iloc[:, 0:3] = testdata.iloc[:, 0:3] * 9.8
    testdata.iloc[:, 3:6] = testdata.iloc[:, 3:6] * np.pi / 180

    # testacc = testimu.a_local
    # testaccnet = testimu.net_acc
    # testacc['a_net'] = testaccnet
    # testacc['g'], testacc['-g'] = 1, -1
    # testacc *= 9.8

    # fig, ax = plt.subplots(figsize=(12, 4))
    # testacc.plot(ax=ax, title=f'acceleration of {file_dropdown.value} lower left shank')

    with output:
        clear_output(wait=True)
        display(testdata)
        # print(f' a_net min for first 2.5 sec: {testacc['a_net'][:2500].min()}\n',f'a_net max for first 2.5 sec: {testacc['a_net'][:2500].max()}')
        # display(fig)

    # plt.close(fig)
    return testdata

file_dropdown.observe(update_output, 'value')
testdata = update_output()

Estimated Initial Orientation (quaternion): [-0.02967247  0.03952791 -0.28907834  0.95602866]


Dropdown(description='Participant:', options=('A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'C03', 'C04', 'C05', '…

Dropdown(description='File:', options=('A01_Fast_01', 'A01_Fast_02', 'A01_Fast_03', 'A01_Fast_04', 'A01_Fast_0…

Output()

In [32]:
csv_path = f'trialData/{participant_dropdown.value}/{file_dropdown.value}.csv'
testdf = pd.read_csv(csv_path, index_col=0)
testimu = imuData('test', testdf, 4)
testdata = testimu.measurements
testdata.iloc[:, 0:3] = testdata.iloc[:, 0:3] * 9.8
testdata.iloc[:, 3:6] = testdata.iloc[:, 3:6] * np.pi / 180

# Generate synthetic or real calibration data
acc_vecs = testdata.iloc[:5, :3].to_numpy()
display(pd.DataFrame(acc_vecs))
orientations = vec2quat_batch(acc_vecs)
# display(px.scatter(orientations))
display(pd.DataFrame(orientations))
pd.DataFrame([vec2quat(vec)[0] for vec in acc_vecs])

,0,1,2
0,-0.942676,-9.680371,-1.622168
1,-0.942676,-9.680371,-1.622168
2,-0.942676,-9.680371,-1.622168
3,-0.942676,-9.680371,-1.622168
4,-0.954471,-9.751140,-1.533707


,0,1,2,3
0,0.763057,-0.634524,0.061790,0.0
1,0.763057,-0.634524,0.061790,0.0
2,0.763057,-0.634524,0.061790,0.0
3,0.763057,-0.634524,0.061790,0.0
4,0.759820,-0.639256,0.062572,0.0


,0,1,2,3
0,0.751803,-0.073211,0.0,0.655310
1,0.751803,-0.073211,0.0,0.655310
2,0.751803,-0.073211,0.0,0.655310
3,0.751803,-0.073211,0.0,0.655310
4,0.749447,-0.073358,0.0,0.657988


In [12]:
plot_trc(f'trcFiles/{participant_dropdown.value}/{file_dropdown.value}.trc')